In [1]:
from __future__ import division, print_function, absolute_import
import logging
import tensorflow as tf
import os
tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
import time

from config import pathDict
from conv_net.train import Train
from conv_net.test import Test
from data_transformation.data_prep import get_valid_land_house_ids, dumpStratifiedBatches_balanced_class
from plot import Plot
import pandas as pd
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")


images_per_label = None # normally 5000 each label is good
assessor_img_type = 'assessor'
aerial_img_type = 'google_aerial' # 'bing_aerial'
overlayed_img_type = 'google_overlayed'
streetside_img_type = None


image_type = overlayed_img_type

if image_type == 'assessor':
    inp_image_shape = [260, 260, 3]
elif image_type == 'google_aerial':
    inp_image_shape = [400, 400, 3]
elif image_type == 'google_overlayed':
    inp_image_shape = [400, 400, 3]
elif image_type == 'google_streetside':
    inp_image_shape = [260, 260, 3]
else:
    raise ValueError('Not a valid image type provided')
    
which_net = 'convnet'
batch_prepare = True
train = True
test = True

## PREPARE/DUMP BATCHES:

In [3]:
start_time = time.time()

cmn_land_pins, cmn_house_pins = get_valid_land_house_ids(
        aerial_img_type=aerial_img_type,
        streetside_img_type=streetside_img_type,
        overlayed_img_type=overlayed_img_type,
        images_per_label=images_per_label)
print (len(cmn_land_pins), len(cmn_house_pins))



tr_batch_size = 128
max_batches = 3
ts_batch_size = (len(cmn_land_pins) + len(cmn_house_pins)) // 10
cv_batch_size = (len(cmn_land_pins) + len(cmn_house_pins)) // 10

dumpStratifiedBatches_balanced_class(cmn_land_pins, cmn_house_pins, img_resize_shape=inp_image_shape,
                                     image_type=image_type, ts_batch_size=ts_batch_size, 
                                     cv_batch_size=cv_batch_size, tr_batch_size=tr_batch_size,
                                     shuffle_seed=873, get_stats=True, max_batches=max_batches)

print ('--------------- %s seconds ------------------'%(time.time() - start_time))

C:\Users\newline\Documents\ImageClassification\data\input_images\aerial_images\google\land
aerial pins:  5433 5433
common aerial pins:  5433 5433
C:\Users\newline\Documents\ImageClassification\data\input_images\overlayed_images\google\land
overlayed pins:  5433 5433
common overlayed pins:  5433 5433
5433 5433
--------------- 237.3948187828064 seconds ------------------


# EVALUATE MODELS:
------------

# TRAIN MODEL: Overlayed Images [Central crop: 96 x 96]
------------

## Train  learning_rate = 0.0001
-----------

In [2]:
max_batches = 3
start_time = time.time()
if train:
    tr_obj = Train(dict(inp_img_shape=[400, 400, 3],
                        crop_shape=[96, 96, 3],
                        out_img_shape=[96, 96, 3],
                        use_checkpoint=True,
                        save_checkpoint=True,
                        write_tensorboard_summary=False
                        ),
                   device_type='gpu',
                   which_net='convnet',  # vgg
                   image_type='google_overlayed')
    (tr_loss_arr, tr_acc_arr, tr_precision_arr, tr_recall_arr,
     cv_loss_arr, cv_acc_arr, cv_precision_arr, cv_recall_arr,
     l_rate_arr) = tr_obj.run(num_epochs=3, num_batches=max_batches, get_stats_at=10)  # + 1)
    
print('----------%s seconds------------'%(str(time.time() - start_time)))

Dumping Checkpoints to %s C:\Users\newline\Documents\ImageClassification\data\data_models\overlayed_images\google\checkpoint\convnet
Dumping Tensorboard Summary to %s C:\Users\newline\Documents\ImageClassification\data\data_models\overlayed_images\google\summary\convnet
Learning Rate: Initial:  0.0005
Checkpoint latest at:  C:\Users\newline\Documents\ImageClassification\data\data_models\overlayed_images\google\checkpoint\convnet\convnet_epoch_2_batch_2
----------41.95160913467407 seconds------------


In [ ]:
max_batches = 3
start_time = time.time()
if train:
    tr_obj = Train(dict(inp_img_shape=[400, 400, 3],
                        crop_shape=[96, 96, 3],
                        out_img_shape=[96, 96, 3],
                        use_checkpoint=True,
                        save_checkpoint=True,
                        write_tensorboard_summary=False
                        ),
                   device_type='cpu',
                   which_net='convnet',  # vgg
                   image_type='google_overlayed')
    (tr_loss_arr, tr_acc_arr, tr_precision_arr, tr_recall_arr,
     cv_loss_arr, cv_acc_arr, cv_precision_arr, cv_recall_arr,
     l_rate_arr) = tr_obj.run(num_epochs=3, num_batches=max_batches, get_stats_at=10)  # + 1)
    
print('----------%s seconds------------'%(str(time.time() - start_time)))

Dumping Checkpoints to %s C:\Users\newline\Documents\ImageClassification\data\data_models\overlayed_images\google\checkpoint\convnet
Dumping Tensorboard Summary to %s C:\Users\newline\Documents\ImageClassification\data\data_models\overlayed_images\google\summary\convnet
Learning Rate: Initial:  0.0005
Checkpoint latest at:  C:\Users\newline\Documents\ImageClassification\data\data_models\overlayed_images\google\checkpoint\convnet\convnet_epoch_5_batch_2
